In [1]:
import os, sys 
%pwd

'd:\\collab_projects\\Brain-Tumor-Detection-using-MLflow-DVC\\notebooks'

In [2]:
os.chdir("../")
%pwd

'd:\\collab_projects\\Brain-Tumor-Detection-using-MLflow-DVC'

In [3]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str 
    local_data_file: Path 
    unzip_dir: Path

In [4]:
from brain_tumor.constants import * 
from brain_tumor.utils.main_utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os 
import urllib.request as request 
import zipfile
import gdown 
from brain_tumor import logger
from brain_tumor.utils.main_utils import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self) -> str:
        """
        Fetch data from the gdrive
        """

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from: {dataset_url} into file: {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from: {dataset_url} into file: {zip_download_dir}")

        except Exception as e:
            raise e 
        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-02 22:33:58,624: INFO: main_utils: yaml file: config\config.yaml loaded successfully]
[2023-10-02 22:33:58,644: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2023-10-02 22:33:58,645: INFO: main_utils: created directory at: artifacts]
[2023-10-02 22:33:58,647: INFO: main_utils: created directory at: artifacts/data_ingestion]
[2023-10-02 22:33:58,648: INFO: 3164952288: Downloading data from: https://drive.google.com/file/d/1bydi7P5-XX96pNSZH8g25SCMZeHFuydU/view?usp=sharing into file: artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1bydi7P5-XX96pNSZH8g25SCMZeHFuydU
To: d:\collab_projects\Brain-Tumor-Detection-using-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 25.6M/25.6M [00:02<00:00, 11.3MB/s]

[2023-10-02 22:34:12,633: INFO: 3164952288: Downloaded data from: https://drive.google.com/file/d/1bydi7P5-XX96pNSZH8g25SCMZeHFuydU/view?usp=sharing into file: artifacts/data_ingestion/data.zip]
